In [ ]:
# !pip install chromadb
# !pip install -Uq chromadb numpy datasets
!pip install langchain
# !pip install openai
# !pip install pypdf
# !pip install python-dotenv
# !pip install tiktoken
# !pip install numpy==1.22
# !pip install faiss-gpu

In [ ]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = [PyPDFLoader("bhasha.pdf")]
docs = []
for load in loader:
  docs.extend(load.load())

In [ ]:
docs[0].page_content

'Bhasha Tyagi +919667314411\nBachelor of Technology tt1222154@iitd.ac.in\nin Textile Engineering bhashatyagi.iitdelhi@gmail.com\nIndian Institute Of Technology, Delhi linkedin.com.in.bhasha-tyagi\nEducation\nDegree Institute/Board CGPA/Percentage Year\nBachelor of Technology Indian Institute of Technology, Delhi 7.05 2022-26\nSenior Secondary Delhi Public School, R.K. Puram Education 96.20 2021\nSecondary Seth Anandram Jaipuria School 97.80 2019\nScholastic Achievements\n•Secured99.29 Percentile in IIT JEE, 2022 with an All India Rank of 6716 among 1.2 million candidates appeared\n•Five-Time Consecutive Medal Recipient of National French Olympiad conducted by Education & Beyond\n•Winner of Saviours Olympiad , and instrumental in School winning the Green Award in environmental initiatives\n•Successfully advanced the Intermediate Round of theInternational Space Olympiad inSuper Senior Category\n•Triple recipient of (B1) French Word Power medals (State and National level) from Prayatna Ed

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 50
)

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
persist_directory = 'docs/chroma/'

In [ ]:
from langchain.vectorstores.faiss import FAISS

vectordb = FAISS.from_documents(splits, embedding)

In [ ]:
vectordb.similarity_search("what are the skills")

[Document(page_content='android app development \nreact \nﬂask \nlangchain \ntensorﬂow \npandas \nNLP \noops \ngit \nnumpy \ndeep learning \nﬁrebase \nleadership \ndsa \nProblem solving \nPresentation skill \nPERSONAL PROJECTS \nAI ResumeRover\n (09/2023)\n \nFunctionality: A platform for HR to upload and manage hundreds of', metadata={'source': 'rohit.pdf', 'page': 0}),
 Document(page_content='languages, I love exploring the power of data to create innovative solutions. \nrohitcode005@gmail.com \n+91 9798494187 \nwww.kaggle.com/rohitcode123 \nlinkedin.com/in/rohit-kumar-66522518a \ngithub.com/Rohitw3code \nEDUCATION \nB.Tech Artiﬁcial Intelligence \nSarala Birla University \nClass 12th', metadata={'source': 'rohit.pdf', 'page': 0}),
 Document(page_content='Rohit Kumar \nData Engineer \nI am Rohit Kumar, a passionate Data Scientist with a strong background in Artiﬁcial Intelligence. Skilled in\nData Science, Machine Learning, Computer Vision, App and Web Development, and various progra

In [ ]:
# from langchain.vectorstores import Chroma
# vectordb = Chroma.from_documents(
#     documents=splits,
#     embedding=embedding,
#     persist_directory=persist_directory
# )


In [ ]:
from langchain.prompts import PromptTemplate

# Build
template = """Use context to answer the question
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.

{context}"
Question: {question}
Helpful Answer:"""

In [ ]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    # chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    return_source_documents=True
)

In [ ]:
result = qa_chain({"query": "provide me the phone number "})
result["result"]

'The phone number provided in the context is +91 9798494187.'